# SQL ConstantCare® Population Report, Spring 2020
May 30, Brent Ozar - [Get the latest version free here](https://www.brentozar.com/archive/category/sql-constantcare/sql-server-population-report/)

<a href="https://www.brentozar.com/sql-constantcare/"><img style="float: right;" src="https://www.brentozar.com/wp-content/uploads/2018/10/SQL_ConstantCare_250x250.png"></a>

[Our SQL ConstantCare® service](https://www.brentozar.com/sql-constantcare/) lets users send data in about their SQL Servers and get health & performance advice via email. Users can opt in to share their anonymized data to get better recommendations overall.

**This data isn't a full picture of the SQL Server ecosystem.** Lemme be really clear: this is by no means a random sample of the full population. It's just a picture of the kinds of servers where people want an inexpensive second opinion. It doesn't include giant corporations, hospitals with PII, or tiny businesses who just shove a SQL Server in the corner and then never take a second glance at it (and if my experience is any indication, never back it up.)

Even though it isn't a full picture, with 3,878 database servers sharing data as of May 30 (up from 3,570 in the last report), there's still a lot of useful information in here for the community. Understanding what servers are out there will help you put context around where your own servers rank, and helps the community better build training material that's relevant for most users.

This population report is just my attempt to help clear up the fog of war. Microsoft simply doesn't give us any adoption information (even though [SQL Server phones home by default](https://www.brentozar.com/archive/2016/06/sql-server-2016-internet-forced-updates-phoning-home/).) If you believed what you saw onstage at conferences and in Microsoft's changing certification programs, you'd think no one still bothers with self-hosted SQL Server, but the few who do have already migrated to SQL Server 2019 running on Kubernetes and Linux. This report is my humble attempt to keep Microsoft honest.

Let's start with a simple question:

## What Versions are the Most Popular?

In [35]:
SELECT 
    CASE WHEN props.engine_edition = 5 THEN 'Azure SQL DB'
        WHEN props.engine_edition = 6 THEN 'Azure Synapse Analytics'
        WHEN props.engine_edition = 8 THEN 'Azure SQL DB MI'
        WHEN props.engine_edition = 9 THEN 'Azure SQL Edge'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 10 AND SPLIT_PART(product_version, '.', 2)::INTEGER = 0 THEN '2008'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 10 AND SPLIT_PART(product_version, '.', 2)::INTEGER = 50 THEN '2008 R2'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 11 THEN '2012'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 12 THEN '2014'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 13 THEN '2016'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 14 THEN '2017'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 15 THEN '2019'
        ELSE 'SQL Server Futurama' END AS SQL_Server_Version,
    SUM(1) AS Servers
FROM collection_server vcs
INNER JOIN registered_user u ON vcs.user_id = u.user_id
INNER JOIN server_property       props ON  vcs.connection_id = props.connection_id
LEFT OUTER JOIN collection_server vcsParent ON vcs.connection_id = vcsParent.prev_connection_id
WHERE vcsParent.run_id IS NULL /* No subsequent collections */
    AND u.email NOT LIKE '%brentozar.com'
    AND vcs.collection_date > (now() - interval '4 week')  /* Only get collections from the last month */
GROUP BY 1
ORDER BY 1
;

(9 row(s) affected)

Total execution time: 00:00:00.783

sql_server_version,servers
2008,68
2008 R2,255
2012,627
2014,694
2016,1293
2017,786
2019,102
Azure SQL DB,51
Azure SQL DB MI,4


Woohoo! This quarter, SQL Server 2019 adoption outpaced SQL Server 2008!

Thoughts on reading that chart:

* SQL Server 2019 still hasn't caught up to 2008R2.
* If you add together 2008 & 2008R2, almost 10% of the servers out there are no longer under support. (That number is really slightly higher since we don't monitor 2000 or 2005, and every time I talk versions in front of a group, folks talk about the 2000 and 2005 instance they still have cowering in shame in a closet somewhere.
* SQL Server 2017 adoption continues to rise, but frankly, 2016 is still the complete juggernaut: 1 in 3 servers is 2016.

I'm going to continue publishing the numbers in this format (only showing the current population) for the first year of the survey, but on the first year's anniversary, I'll switch to graphs that show trending and year-over-year changes.

How about development? Are people maybe testing their apps on newer versions?



## What Versions are People Using in Development?


In [37]:
SELECT 
    CASE WHEN props.engine_edition = 5 THEN 'Azure SQL DB'
        WHEN props.engine_edition = 6 THEN 'Azure Synapse Analytics'
        WHEN props.engine_edition = 8 THEN 'Azure SQL DB MI'
        WHEN props.engine_edition = 9 THEN 'Azure SQL Edge'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 10 AND SPLIT_PART(product_version, '.', 2)::INTEGER = 0 THEN '2008'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 10 AND SPLIT_PART(product_version, '.', 2)::INTEGER = 50 THEN '2008 R2'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 11 THEN '2012'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 12 THEN '2014'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 13 THEN '2016'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 14 THEN '2017'
        WHEN SPLIT_PART(product_version, '.', 1)::INTEGER = 15 THEN '2019'
        ELSE 'SQL Server Futurama' END AS SQL_Server_Version,
    SUM(1) AS Servers
FROM collection_server vcs
INNER JOIN registered_user u ON vcs.user_id = u.user_id
INNER JOIN server_property       props ON  vcs.connection_id = props.connection_id
LEFT OUTER JOIN collection_server vcsParent ON vcs.connection_id = vcsParent.prev_connection_id
WHERE vcsParent.run_id IS NULL /* No subsequent collections */
    AND u.email NOT LIKE '%brentozar.com'
    AND vcs.collection_date > (now() - interval '4 week')  /* Only get collections from the last month */
    AND props.edition LIKE 'Developer%'
GROUP BY 1
ORDER BY 1
;

(7 row(s) affected)

Total execution time: 00:00:00.779

sql_server_version,servers
2008,2
2008 R2,10
2012,39
2014,38
2016,96
2017,47
2019,33


SQL Server 2019's adoption rate for development servers doubled from the last quarter, and that's a great sign. People are starting to develop more against 2019. However, 2017's growth wasn't as strong - I think it basically means people are using 2019 as their default new dev box, but it doesn't bode well for a rapid adoption of 2019 in production over the next couple of quarters. People just aren't testing on it yet, and testing is utterly critical for 2019 given its radical differences in execution plans (batch mode on rowstore, inlined scalar functions, etc.)

I think about this chart a lot when I'm designing training. When is the right time to start building developer training material for SQL Server 2019? It's a tricky timing act: you need to be skilled-up before you adopt the new version, and I need to have the training material ready to go before then.

However, if I focus on building 2019-specific training material right now, it could be 6-12-18 months before it really starts to pay off. Judging by this chart, I'd be better off building 2017-specific training material since there are a lot of people who still haven't adopted it yet. I use 2019 in my class lab VMs, but I still focus on 2017 compat mode (and only require the bring-your-own-VM students to use 2017 minimum, not 2019.)

For the rest of the census report, I'm going to focus on just the production servers.

## How Much Data do Servers Have?

For quick, easy reporting (and because accuracy here has relatively limited value), I'm going to use user database data file sizes, not contents, and not log files.

In [39]:
WITH IndividualServers AS (
    SELECT vcs.connection_id,
        CASE WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) < 25 THEN '<25'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 125 THEN '25-125'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 225 THEN '125-225'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 325 THEN '225-325'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 425 THEN '325-425'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 525 THEN '425-525'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 625 THEN '525-625'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 725 THEN '625-725'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 825 THEN '725-825'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 925 THEN '825-925'
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 1025 THEN '925-1025'
        ELSE '>1025' END AS Size_Category,

        CASE WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) < 25 THEN 1
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 125 THEN 2
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 225 THEN 3
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 325 THEN 4
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 425 THEN 5
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 525 THEN 6
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 625 THEN 7
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 725 THEN 8
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 825 THEN 9
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 925 THEN 10
        WHEN ROUND(SUM(pcData.cntr_value / 1024.0 / 1024 )::numeric, 3) <= 1025 THEN 11
        ELSE 12 END AS Size_Category_Id


    FROM collection_server vcs
    INNER JOIN registered_user u ON vcs.user_id = u.user_id
    INNER JOIN server_property       props ON  vcs.connection_id = props.connection_id
    INNER JOIN sys_dm_os_performance_counters pcData ON pcData.connection_id = vcs.connection_id
            AND    pcData.counter_name = 'Data File(s) Size (KB)'
            AND    pcData.instance_name <> '_Total'
            AND    pcData.instance_name <> 'master'
            AND    pcData.instance_name <> 'model'
            AND    pcData.instance_name <> 'msdb'
            AND    pcData.instance_name <> 'tempdb'
    LEFT OUTER JOIN collection_server vcsParent ON vcs.connection_id = vcsParent.prev_connection_id
    WHERE vcsParent.run_id IS NULL /* No subsequent collections */
        AND u.email NOT LIKE '%brentozar.com'
        AND vcs.collection_date > (now() - interval '4 week')  /* Only get collections from the last month */
        AND props.edition NOT LIKE 'Developer%'
    GROUP BY vcs.connection_id
)
SELECT Size_Category AS Data_File_Size_GB, SUM(1) AS Servers
FROM IndividualServers
GROUP BY Size_Category, Size_Category_Id
ORDER BY Size_Category_Id;

(12 row(s) affected)

Total execution time: 00:00:01.152

data_file_size_gb,servers
<25,1049
25-125,795
125-225,348
225-325,218
325-425,175
425-525,116
525-625,80
625-725,65
725-825,62
825-925,41


Keep in mind that this is the total data file size for all user databases on the server, and these numbers really help to keep the world in perspective.

Over half of all servers host less than 225GB data.

Over 2/3 of servers are smaller than the Stack Overflow demo database, and that doesn't even have nonclustered indexes. That's why I love using that database for training: it challenges a very good chunk of the audience.

Consistent with the last population report, about 15% have a terabyte or more of data, with some of them having dozens of terabytes. That's useful because it gives me over 500 servers' worth of metadata to help guide folks in larger tiers. Performance advice is dramatically different at that scale: it's so much harder and more expensive to throw memory and indexes at problems. Heck, just creating an index on a 1TB table can represent a big challenge for some shops.

# How Much Hardware Do Production Servers Get?

In [40]:
WITH IndividualServers AS (
    SELECT vcs.connection_id,
        CASE WHEN SUM(1) <= 4 THEN '1-4'
        WHEN SUM(1) <= 8 THEN '5-8'
        WHEN SUM(1) <= 12 THEN '9-12'
        WHEN SUM(1) <= 16 THEN '13-16'
        WHEN SUM(1) <= 20 THEN '17-20'
        WHEN SUM(1) <= 24 THEN '21-24'
        WHEN SUM(1) <= 28 THEN '25-28'
        WHEN SUM(1) <= 32 THEN '29-32'
        ELSE '>=33' END AS Size_Category,

        CASE WHEN SUM(1) <= 4 THEN 1
        WHEN SUM(1) <= 8 THEN 2
        WHEN SUM(1) <= 12 THEN 3
        WHEN SUM(1) <= 16 THEN 4
        WHEN SUM(1) <= 20 THEN 5
        WHEN SUM(1) <= 24 THEN 6
        WHEN SUM(1) <= 28 THEN 7
        WHEN SUM(1) <= 32 THEN 8
        ELSE 9 END AS Size_Category_Id

    FROM collection_server vcs
    INNER JOIN registered_user u ON vcs.user_id = u.user_id
    INNER JOIN server_property       props ON  vcs.connection_id = props.connection_id
    INNER JOIN sys_dm_os_schedulers cpu ON vcs.connection_id = cpu.connection_id
            AND    cpu.status IN('VISIBLE ONLINE','VISIBLE OFFLINE')
    LEFT OUTER JOIN collection_server vcsParent ON vcs.connection_id = vcsParent.prev_connection_id
    WHERE vcsParent.run_id IS NULL /* No subsequent collections */
        AND u.email NOT LIKE '%brentozar.com'
        AND vcs.collection_date > (now() - interval '4 week')  /* Only get collections from the last month */
        AND props.edition NOT LIKE 'Developer%'
    GROUP BY vcs.connection_id
)
SELECT Size_Category AS CPU_Cores, SUM(1) AS Servers
FROM IndividualServers
GROUP BY Size_Category, Size_Category_Id
ORDER BY Size_Category_Id;

(9 row(s) affected)

Total execution time: 00:00:00.712

cpu_cores,servers
1-4,1687
5-8,786
9-12,187
13-16,278
17-20,92
21-24,134
25-28,1
29-32,306
>=33,91


Again, really helps set expectations: the majority of servers out there are pretty doggone small. You can go a really, really long way with just a few CPU cores, especially if you're diligent about query tuning. At $2,000 per core for Standard Edition, you wanna avoid throwing cores at a problem if you can avoid it.

About 2/3 of the servers have less CPU power than a modern pro-grade laptop. That's not a number to be ashamed of - that's a number to celebrate! You can do a lot with a little. As with last quarter's report, only about 10% of servers have over 24 cores.

## How Much Memory Does the Server OS Have?

Memory isn't licensed - although Standard Edition has a ~128GB cap - so how much memory do folks have at the OS level:

In [41]:
WITH IndividualServers AS (
    SELECT vcs.connection_id,
        CASE WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 15 THEN '0-15'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 31 THEN '16-31'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 47 THEN '32-47'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 63 THEN '48-63'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 79 THEN '64-79'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 95 THEN '80-95'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 111 THEN '96-111'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 127 THEN '112-127'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 143 THEN '128-143'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 159 THEN '144-159'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 175 THEN '160-175'
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 192 THEN '176-192'
        ELSE '>192' END AS Size_Category,

        CASE WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 15 THEN 0
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 31 THEN 1
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 47 THEN 2
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 63 THEN 3
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 79 THEN 4
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 95 THEN 5
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 111 THEN 6
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 127 THEN 7
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 143 THEN 8
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 159 THEN 9
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 175 THEN 10
        WHEN ROUND((os.total_physical_memory_kb / 1024.0 / 1024)::numeric,0) <= 192 THEN 11
        ELSE 12 END AS Size_Category_Id

    FROM collection_server vcs
    INNER JOIN registered_user u ON vcs.user_id = u.user_id
    INNER JOIN server_property       props ON  vcs.connection_id = props.connection_id
    INNER JOIN sys_dm_os_sys_memory os ON vcs.connection_id = os.connection_id
    LEFT OUTER JOIN collection_server vcsParent ON vcs.connection_id = vcsParent.prev_connection_id
    WHERE vcsParent.run_id IS NULL /* No subsequent collections */
        AND u.email NOT LIKE '%brentozar.com'
        AND vcs.collection_date > (now() - interval '4 week')  /* Only get collections from the last month */
        AND props.edition NOT LIKE 'Developer%'
)
SELECT Size_Category AS Memory_OS_GB, SUM(1) AS Servers
FROM IndividualServers
GROUP BY Size_Category, Size_Category_Id
ORDER BY Size_Category_Id;

(13 row(s) affected)

Total execution time: 00:00:00.407

memory_os_gb,servers
0-15,701
16-31,726
32-47,456
48-63,223
64-79,345
80-95,48
96-111,111
112-127,40
128-143,233
144-159,46


I'm specifically looking at memory for the entire server OS here, like how much Windows sees.

There are several ways you can look at SQL Server memory, like target (how much it wants to use) or total (how much it's currently using.) I kept it simple and I'm just looking at OS memory because I want to get a feel for the hardware population overall, not how people are configuring SQL Server, or [whether their workloads are causing total memory to rise to the target counter](https://www.brentozar.com/archive/2018/05/how-to-tell-if-your-sql-server-has-too-much-memory/).

When you think about these metrics, remember that over half of all production servers host <225GB data files. If you handed me a server with, say, 200GB of data and asked me how much memory to assign it, I think 32-48GB is actually a pretty realistic number. I might even call it generous - keeping in mind that RAM is still ridiculously overpriced in the cloud, and it's tied to core counts.

Now, look at the metrics: half of the servers host <225GB of data, and half of servers have <48GB RAM. That makes decent sense. 0-15GB RAM sounds terrifying, but remember that 1,049 servers are hosting <25GB of data.

When you're at these really low database sizes, I can see why people would think they'd be interested in a container-based solution: the overhead of 1,049 servers running Windows is pretty ugly. Containers might remove some of that overhead.

## Bonus: What Unusual Wait Types are Showing Up?

One of the fun things with gathering performance metrics on thousands of servers is that we get to see some pretty nifty edge cases. Here are wait stats that have only been seen by a handful of people in the last 30 days:

In [48]:
SELECT wait_type, COUNT(DISTINCT user_id) AS users_who_saw_it, SUM(waiting_tasks_count) AS waiting_tasks,
    SUM(wait_time_ms / 1000.0 / 60) AS wait_time_minutes, AVG(avg_wait_time_ms * 1.0) AS avg_wait_time_ms
FROM wait_stats w
GROUP BY wait_type
HAVING COUNT(DISTINCT user_id) <= 5
ORDER BY avg_wait_time_ms DESC;

(110 row(s) affected)

Total execution time: 00:00:11.501

wait_type,users_who_saw_it,waiting_tasks,wait_time_minutes,avg_wait_time_ms
HADR_FABRIC_CALLBACK,2,10597,125280.8398166666666668,818345.263888888889
PVS_PREALLOCATE,2,36506,125334.7800833333333335,498484.319444444444
HADR_SEEDING_LIMIT_BACKUPS,2,36,256.21445000000000003333,368789.333333333333
XTP_TRUNCATION_LSN,3,14,140.985399999999999966656667,352466.583333333333
PWAIT_ALL_COMPONENTS_INITIALIZED,2,39,54.63303333333333280005,84104.684210526316
RESTORE_SERVICE_CALL,1,20,12.97506666666666670000,52959.285714285714
CLR_JOIN,5,18676,471.89568333333333310000,23027.379629629630
SEMPLAT_DSI_BUILD,1,1,0.17093333333333333333,10256.0000000000000000
SLEEP_MSDBSTARTUP,1,1,0.09305000000000000000,5583.0000000000000000
SATELLITE_SERVICE_SETUP,4,7081,54.37826666666666656668,2478.0714285714285714
